# Key Setup

In [ ]:
import random
import math
from sympy import isprime

In [ ]:
def generate_16_bit_primes():
  prime_1 = 1 #initializing with a non prime number
  while not isprime(prime_1):
    prime_1 = random.randrange(32768,65536)

  prime_2 = 1 #initializing with a non prime number
  while not isprime(prime_2) or prime_1==prime_2:
    prime_2 = random.randrange(32768,65536)
  return prime_1, prime_2

def generate_gcd(a,b):
  while b:
    a, b = b, a % b
  return a

def generate_exponent(phiN):
  exponent = random.randrange(2, phiN)
  while generate_gcd(exponent,phiN) != 1:
    exponent = random.randrange(2, phiN)
  return exponent

def generate_modular_inverse(num1, num2):
  var1,var2 = 0,1
  m = num2

  while num1 > 1:
    res = num1 // num2
    num2, num1 = num1 % num2, num2
    var1, var2 = var2 - res * var1, var1

  return var2 + m if var2 < 0 else var2

In [ ]:
p,q = generate_16_bit_primes()
n = p*q
phiN = (p-1)*(q-1)
e = generate_exponent(phiN)
d = generate_modular_inverse(e,phiN)

print(f"P: {p}\nQ: {q}\ne: {e}\nN: {n}\nd: {d}\n")
print(f"N,e: ({n},{e})")
print(f"N,d: ({n},{d})")

P: 51407
Q: 65071
e: 265872071
N: 3345104897
d: 3311748311

N,e: (3345104897,265872071)
N,d: (3345104897,3311748311)


In [19]:
P = 51407
Q = 65071
print((P-1)*(Q-1))

3344988420


# Encryption and Decryption

In [48]:
def generate_modular_multiplication(mod, base, result=None):
  if result is None:
    result = base
  return (result*base)%mod

def generate_modular_square_multiplication(base, exponent, modulus):
  result = 1
  while exponent > 0:
    if (exponent % 2):
      result = generate_modular_multiplication(modulus,base,result)
    base = generate_modular_multiplication(modulus,base)
    exponent //= 2

  return result

In [49]:
def generate_text_to_int_msg_list(text, length):
  print([text[i:i+length] for i in range(0, len(text), length)])
  return [int(text[i:i+length].encode('utf-8').hex(),16) for i in range(0, len(text), length)]

In [50]:
def generate_encrypted_list(list,e,N):
  return [generate_modular_square_multiplication(text,e,N) for text in list]

## For encryption

In [51]:
#Partner's N & e
N = 2476210577
e = 1001395693

my_message = "Hello From Anik"
block_len = 3
print(my_message)
msg_list = generate_text_to_int_msg_list(my_message,block_len)
print("message before encryption: ")
print(msg_list)
encrypted_msg_list = generate_encrypted_list(msg_list,e,N)
print("message after encryption: ")
print(encrypted_msg_list)

Hello From Anik
['Hel', 'lo ', 'Fro', 'm A', 'nik']
message before encryption: 
[4744556, 7106336, 4616815, 7151681, 7235947]
message after encryption: 
[954186234, 640257641, 504336221, 2241249811, 902176818]


## For Decryption

In [52]:
#My N & d
N = 3345104897
d = 3311748311
encrypted_list_from_partner = [2168000973, 1524214925, 3070310645, 2531220278]

print("message after decryption: ")
decrypted_int_list = generate_encrypted_list(encrypted_list_from_partner,d,N)
decrypted_msg_list = [bytes.fromhex(hex(int_msg)[2:]).decode('utf-8') for int_msg in decrypted_int_list]
print(decrypted_int_list)
print(decrypted_msg_list)
print("".join(decrypted_msg_list))


message after decryption: 
[4744556, 7106336, 4288105, 107]
['Hel', 'lo ', 'Ani', 'k']
Hello Anik


# Signature

In [53]:
def generate_signed_original_list(list,e_d,N):
  return [generate_modular_square_multiplication(text,e_d,N) for text in list]

def is_verified(original_text, signature_received):
  return True if original_text == "".join([bytes.fromhex(hex(int_val)[2:]).decode('utf-8') for int_val in signature_received]) else False


## Signature Generation

In [54]:
N = 3345104897
d = 3311748311
original_text = "Anik Chowdhury"
text_lst = generate_text_to_int_msg_list(original_text, 3)
print(f"Original Text: {text_lst}")
signed_text = generate_signed_original_list(text_lst,d,N)
print(f"Signed Text: {signed_text}")

['Ani', 'k C', 'how', 'dhu', 'ry']
Original Text: [4288105, 7020611, 6844279, 6580341, 29305]
Signed Text: [1532896188, 1727526906, 730877332, 607670896, 43522356]


## Signature Verification

In [55]:
#Partner's N & e
N = 2476210577
e = 1001395693
partners_signature = "Pushpeswaree Degamber"
partners_signature_received = [1506823430, 2438002494, 1358545013, 710806339, 611410040, 1284107460, 754649663]
partner_original_text = generate_signed_original_list(partners_signature_received,e,N)
print(f"Partner's Text: {partner_original_text}")
is_verified(partners_signature, partner_original_text)

Partner's Text: [5272947, 6844517, 7567201, 7497061, 2114661, 6775149, 6448498]


True